In [9]:
import polars as pl
import altair as alt
from pre_process import filter_regions, process_data, filter_south_asia, long_acc_ownership_df, savings_behavior_df

In [10]:
south_asia_countries = ['South Asia', 'India', 'Bangladesh', 'Pakistan']

In [11]:
region_order = ['South Asia','Sub-Saharan Africa', 'Middle East & North Africa',
                'Latin America & Caribbean', 'Europe & Central Asia', 'East Asia & Pacific']

In [10]:
def region_any_account(df):
    # here I used ChatGPT to help me set the labels for the legens from 0-100 instead of
    # the min and max values in the data.
    """
    Generates a heat map for account ownership (bank and mobile money) for 
    regions

    Input: pl.DataFrame: A DataFrame
    Return: alt.Chart: The generated Altair heat map.
    """
    
    subtitle_lines = [
        'The percentage of respondents who report having an account at a bank',
        'or similar financial institution or report personally using',
        'a mobile money service in the past year.']
    
    any_account_chart = alt.Chart(df).mark_rect().encode(
        x = 'year:N', 
        y = alt.Y('clean_region_name', title = 'Region', sort = region_order), 
        color = alt.Color('account_t_d', title = '% Adults (age 15+)', 
                          scale=alt.Scale(domain=[0, 100], clamp=True, scheme ='goldgreen' ))
    ).properties(
        title = alt.TitleParams(
            text = 'Account Ownership (2011-2024)',
            subtitle = subtitle_lines ,
            subtitleColor='gray',
            subtitleFontSize=12, 
            align = 'left')).configure_title(
            anchor='start')
    return any_account_chart


# save as svg
region_any_account(filter_regions(process_data())).save('../milestones/static-draft-images/region_any_account.svg', 


alt.Chart(...)

In [14]:
def region_fin_resiliance(df): 
    """
    Generates a bar chart to show financial resilience for each region

    Input: pl.DataFrame: A DataFrame
    Return: alt.Chart: The generated Altair bar chart.
    """
    
    fin_resiliance_df = df.filter(pl.col('year') != 2021)

    subtitle_lines = [
        'The percentage of respondents who say it is possible and',
        'not difficult at all or somewhat difficult to come up',
        'with the funds in 30 days.'
    ]

    chart = alt.Chart(fin_resiliance_df).mark_bar().encode(
        x = alt.X('clean_region_name:N', title = '', sort= '-y',  axis=alt.Axis(labels=False, ticks=False, domain=False)),
        y = alt.Y('fin24aSD_ND:Q', title = '% Adults (age 15+)'),
        color = alt.Color('clean_region_name',title = 'Region').scale(scheme = 'set2'),
        ).properties(
        title = alt.TitleParams(
            text = 'Financial Resilience (2024)',
            subtitle = subtitle_lines,
            subtitleColor='gray',
            subtitleFontSize=12,
            align = 'left')).configure_title(
            anchor='start')

    return chart

# save as svg
region_fin_resiliance(filter_regions(process_data())).save('../milestones/static-draft-images/region_fin_resiliance_new.svg',
                                                           ppi=200)
    

In [11]:
def region_savings(df): 
    """
    Generates a connected dot plot to show percentage of respondented who saved using formal methods
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated Altair dot plot.
    """

    savings_df = df.filter((pl.col('year') == 2011) | (pl.col('year') == 2024))

    subtitle_lines = [
        'The percentage of respondents who saved at a bank or similar financial institution', 
        'or using a mobile money account in the past year.'
    ]

    region_savings = alt.Chart(savings_df).mark_line(point=True).encode(
        y = alt.Y('clean_region_name:N', title = '', sort = region_order),
        x = alt.X('fin17a_17a1_d:Q', title = '% Adults (age 15+)'),
        color = alt.Color('clean_region_name', legend = None).scale(scheme = 'set2'),
        shape = alt.Shape('year:O', title = 'Year', scale=alt.Scale(
            domain=[2011,2024],
            range=['circle', 'triangle-up']
              ))
    ).properties(
        title = alt.TitleParams(
            text = 'Financial use: formal savings (2011-2024)',
            subtitle = subtitle_lines,
            subtitleColor = 'gray',
            subtitleFontSize=12,
            align = 'left')).configure_title(
            anchor='start')


    return region_savings


# save as svg
region_savings(filter_regions(process_data())).save('../milestones/static-draft-images/region_savings_new.svg', 
                                                             ppi=200)

alt.Chart(...)

In [15]:
def country_account_ownership(df):
    """
    Generates a bar chart for each country to illustrate account ownership
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated Altair bar chart.
    """

    new_colors = ['#A6D854','#984EA3', '#377EB8','#F781BF']

    subtitle_lines = [
        'The percentage of respondents who saved at a bank or similar financial institution',
        'or using a mobile money account in the past year.']


    country_account_chart = alt.Chart(df).mark_bar().encode(
        x='year:N',
        y = alt.Y('account_per:Q', title = '% Adults (age 15+)'),
        color=alt.Color(
              'countrynewwb:N',
              title='Country',
              scale=alt.Scale(domain=south_asia_countries, range=new_colors),
            legend=alt.Legend(orient='bottom'))
          ), 
        xOffset=alt.XOffset('countrynewwb:N', scale=alt.Scale(domain=south_asia_countries))
        ).facet(
    facet=alt.Facet('account_type:N', title='', sort=south_asia_countries),
    columns=2).properties(
        title = alt.TitleParams(
            text = 'Account Ownership By Country (2011-2024)',
            subtitle = 'The percentage of respondents who saved at a bank or similar financial institution or using a mobile money account in the past year',
            subtitleColor = 'gray',
            subtitleFontSize=12,
            align = 'left')).configure_title(
            anchor='start')


    return country_account_chart

# save as svg
country_account_ownership(long_acc_ownership_df(filter_south_asia(process_data(), False)))
country_account_ownership(long_acc_ownership_df(filter_south_asia(process_data(), False))).save('../milestones/static-draft-images/country_account_ownership_new.svg', 
                                                             ppi=200)

In [2]:
def account_ownership_gender_gaps(df):
    """
    Generates a connected dot plot to show account ownership by gender
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated Altair dot plot.
    """
    
    gender_df = df.filter(pl.col('group') == 'gender')
    gender_df_filt = gender_df.filter((pl.col('year') == 2011) | (pl.col('year') == 2024))
    
    new_colors = ['#8624f5', '#1fc3aa']


    subtitle_lines = [
        'The percentage of male or female respondents who saved at a bank',
        'or similar financial institution or using a mobile money account in the past year.'
    ]

    account_gender_gap = alt.Chart(gender_df_filt).mark_line(point = True).encode(
        x = alt.X('account_t_d:Q', title = '% Adults (age 15+)'),
        y = alt.Y('group2:N', title = ''),
        color = alt.Color('group2:N', legend = None,scale = alt.Scale(range = new_colors)),
                shape = alt.Shape('year:O', title = 'Year', scale=alt.Scale(
            domain=[2011,2024],
            range=['circle', 'triangle-up']
              ))

    ).facet(
    facet=alt.Facet('countrynewwb:N', title='Country', sort=south_asia_countries),
    columns=1).properties(
        title = alt.TitleParams(
            text = 'Account Ownership By Gender (2011-2024)',
            subtitle = subtitle_lines,
            subtitleColor = 'gray',
            subtitleFontSize=12, 
            align = 'left')).configure_title(
            anchor='start')

    return account_gender_gap


# save as svg
account_ownership_gender_gaps(filter_south_asia(process_data(), True)).save('../milestones/static-draft-images/any_account_ownership_gender_gaps_new.svg', 
                                                             ppi=200)

NameError: name 'filter_south_asia' is not defined

In [59]:
def account_income_gaps(df):
    """
    Generates a heat map to show account ownership for the poorest 40% and richest 60%
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated Altair heat map.
    """
    subtitle_lines = [
    "The percentage of respondents in each income group",
    "who saved at a bank or similar financial institution",
    "or using a mobile money account in the past year."
]
    
    income_df = df.filter(pl.col('group') == 'income')
    
    account_income_gap = alt.Chart(income_df).mark_rect().encode(
        x = 'year:N', 
        y = alt.Y('group2:N', title = ''), 
        color = alt.Color('account_t_d:Q', title = '% Adults (age 15+)',
                          scale=alt.Scale(domain=[0, 100], clamp=True, scheme ='goldgreen'))    
    ).facet(
    facet=alt.Facet('countrynewwb:N', title='Country', sort=south_asia_countries),
    columns=1).properties(
        title = alt.TitleParams(
            text = 'Account Ownership By Income Group (2011-2024)',
            subtitle = subtitle_lines,
            subtitleColor = 'gray',
            subtitleFontSize=12, 
            align = 'left')).configure_title(
            anchor='start')
    return account_income_gap


# save as svg                         
account_income_gaps(filter_south_asia(process_data(), True)).save('../milestones/static-draft-images/any_account_income_gaps.svg', 
                                                            ppi=200)


In [16]:
def savings_behavior_by_country(df): 
    """
    Generates a connected dot plot to show savings behavior by country over time
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated Altair dot plot
    """
    savings_chart = alt.Chart(df).mark_line(point = True).encode(
        x = 'year:N',
        y = alt.Y('savings_per:Q', title = '% Adults (age 15+)'),
        color = alt.Color('savings_method', title = 'Savings Method', legend=alt.Legend(orient='bottom')).scale(scheme = 'greenblue'),  
        column = alt.Column('countrynewwb', title = 'Country', sort = south_asia_countries)
    
    ).properties(
        title = alt.TitleParams(
            text = 'Savings Behavior By Country (2011-2024)',
            subtitle = 'The percentage of respondents who reported saving formally or informally.',
            subtitleColor = 'gray',
            subtitleFontSize=12, 
            align = 'left')).configure_title(
            anchor='start')
    return savings_chart


# save as svg
savings_behavior_by_country(savings_behavior_df(filter_south_asia(process_data(), False))).save('../milestones/static-draft-images/savings_behavior_by_country_new.svg', 
                                                             ppi=200)

In [57]:
def financial_resiliance_all(df):
    """
    Generates a bar chart to to illustrate financial resilience in 2024 for south asia
    and the relevant countries
    
    Input: pl.DataFrame: A DataFrame
    
    Return: alt.Chart: The generated bar chart
    """
    
    df_filt = df.filter(pl.col('year') == 2024)

    new_colors = ['#A6D854','#984EA3', '#377EB8','#F781BF']

    subtitle_lines = [
        'The percentage of respondents who say it is possible and not difficult at all', 
        'or somewhat difficult to come up with the funds in 30 days.'
    ]

    country_resiliance_chart = alt.Chart(df_filt).mark_bar().encode(
        x = alt.X('countrynewwb:N', sort = south_asia_countries, title = ''),
        y = alt.Y('fin24aSD_ND:Q', title = '% Adults (age 15+)'),
        color=alt.Color(
              'countrynewwb:N',
              title='Country',
              scale=alt.Scale(domain=south_asia_countries, range=new_colors)
          )
    ).properties(
        width = 175,
        title = alt.TitleParams(
            text = 'Financial Resilience (2024)',
            subtitle = subtitle_lines,
            subtitleColor = 'gray',
            subtitleFontSize=12, 
            align = 'left')).configure_title(
            anchor='start')
    return country_resiliance_chart

# save as svg
financial_resiliance_all(filter_south_asia(process_data(), False)).save('../milestones/static-draft-images/financial_resiliance_all_new.svg', 
                                                             ppi=200)